In [17]:
import json
import random
from pycocotools.coco import COCO
import requests
import os

from xml.etree.ElementTree import Element, SubElement, ElementTree

Helper function to make PASCAL VOC-style XML annotations.

In [18]:
# Function to create Pascal-style XML annotations
def create_pascal_xml(img_info, annotations, output_annotation_dir):
    xml_root = Element('annotation')
    folder = SubElement(xml_root, 'folder')
    folder.text = 'VOC_COCO'

    filename = SubElement(xml_root, 'filename')
    filename.text = img_info['file_name']

    size = SubElement(xml_root, 'size')
    SubElement(size, 'width').text = str(img_info['width'])
    SubElement(size, 'height').text = str(img_info['height'])
    SubElement(size, 'depth').text = '3'  # Assuming RGB images

    for ann in annotations:
        obj = SubElement(xml_root, 'object')
        name = SubElement(obj, 'name')
        name.text = 'person'  # Category name

        bndbox = SubElement(obj, 'bndbox')
        bbox = ann['bbox']  # COCO format: [xmin, ymin, width, height]
        x_min = int(bbox[0])
        y_min = int(bbox[1])
        x_max = int(bbox[0] + bbox[2])
        y_max = int(bbox[1] + bbox[3])
        SubElement(bndbox, 'xmin').text = str(x_min)
        SubElement(bndbox, 'ymin').text = str(y_min)
        SubElement(bndbox, 'xmax').text = str(x_max)
        SubElement(bndbox, 'ymax').text = str(y_max)

        # Add keypoints
        keypoints_elem = SubElement(obj, 'keypoints')
        keypoints = ann.get('keypoints', [])
        for i in range(0, len(keypoints), 3):
            kp_x = int(keypoints[i])
            kp_y = int(keypoints[i + 1])
            visibility = int(keypoints[i + 2])  # Visibility flag
            keypoint = SubElement(keypoints_elem, 'keypoint')
            SubElement(keypoint, 'x').text = str(kp_x) if visibility > 0 else 'NaN'
            SubElement(keypoint, 'y').text = str(kp_y) if visibility > 0 else 'NaN'
            SubElement(keypoint, 'visibility').text = str(visibility)

    # Save XML
    output_file = os.path.join(output_annotation_dir, f"{img_info['file_name'].split('.')[0]}.xml")
    tree = ElementTree(xml_root)
    tree.write(output_file)

In [19]:
# downloading 2000 images of training data for category person
# Paths
annotation_file = 'annotations/instances_train2017.json'  # Update with your COCO annotation file path
output_dir = 'coco_output/'  # Folder to save downloaded images
annotations_dir = os.path.join(output_dir, 'Annotations')
sets_dir = os.path.join(output_dir, 'ImageSets', 'Main')
images_dir = os.path.join(output_dir, 'JPEGImages')
# os.makedirs(output_dir, exist_ok=True)

# Number of images to sample
num_images = 10

In [9]:
# Load COCO annotations
coco = COCO(annotation_file)

loading annotations into memory...
Done (t=17.18s)
creating index...
index created!


In [20]:
# Get category ID for 'person'
person_category_id = coco.getCatIds(catNms=['person'])[0]

# Get all image IDs containing persons
person_image_ids = coco.getImgIds(catIds=[person_category_id])

In [16]:
# Randomly sample 2000 images / 4100 instances
sampled_image_ids = random.sample(person_image_ids, num_images)

# Download sampled images
for img_id in sampled_image_ids:
    img_info = coco.loadImgs(img_id)[0]
    img_url = img_info['coco_url']
    img_filename = os.path.join(output_dir, 'JPEGImages', img_info['file_name'])
    
    # Download image
    response = requests.get(img_url, stream=True)
    if response.status_code == 200:
        with open(img_filename, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
    print(f"Downloaded {img_info['file_name']}")

print(f"Downloaded {num_images} person images to {output_dir}")

Downloaded 000000015957.jpg
Downloaded 000000127556.jpg
Downloaded 000000494257.jpg
Downloaded 000000206486.jpg
Downloaded 000000578984.jpg
Downloaded 000000103486.jpg
Downloaded 000000014388.jpg
Downloaded 000000416115.jpg
Downloaded 000000175946.jpg
Downloaded 000000009990.jpg
Downloaded 10 person images to coco_output/
